In [32]:
# First XGBoost model
import numpy as np
import streamlit as st
import sklearn
import datetime
import pytz
import pickle
import re # for regex
import random
from numpy import loadtxt
from xgboost import XGBClassifier
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
import numpy as np
import pandas as pd
import pickle as pk
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from datetime import datetime, date
import calendar
from sklearn import ensemble
from sklearn.model_selection import StratifiedKFold
import sklearn.metrics as metrics
import warnings
warnings.filterwarnings('ignore')
print(xgboost.__version__)

1.5.0


In [33]:
#Load Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Loading the dataset 
currentdf  = pd.read_csv(r"C:\Users\Russell\Desktop\SmartParkingApp\PL80_Dataset_1.csv")
currentdf = currentdf.dropna()
currentdf = currentdf.sample(frac=1)

print(currentdf.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11548 entries, 6771 to 5169
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SESSIONID         11548 non-null  object 
 1   CARPARKNUMBER     11548 non-null  object 
 2   VEHICLETYPE       11548 non-null  object 
 3   SESSIONSTART      11548 non-null  object 
 4   SESSIONEND        11548 non-null  object 
 5   TOTAL_CHARGE      11548 non-null  float64
 6   DURATION          11548 non-null  int64  
 7   EFFECTIVE_CHARGE  11548 non-null  float64
 8   LOT_NUMBER        11548 non-null  int64  
 9   LABEL_TYPE        11548 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 992.4+ KB
None


In [34]:
#y is the output and X is the input to train
y = currentdf['LABEL_TYPE']
X = currentdf.drop(['SESSIONID','CARPARKNUMBER','LABEL_TYPE', 'LOT_NUMBER'], axis = 1)
X.head()

,VEHICLETYPE,SESSIONSTART,SESSIONEND,TOTAL_CHARGE,DURATION,EFFECTIVE_CHARGE
6771,C,18/8/2021 21:20,18/8/2021 22:21,1.22,61,1.22
10923,C,25/10/2021 19:48,25/10/2021 20:08,0.00,20,0.00
5242,C,29/10/2021 19:36,29/10/2021 19:37,0.00,1,0.00
1336,C,10/10/2021 23:01,11/10/2021 9:27,0.00,626,0.00
3978,C,23/10/2021 20:19,24/10/2021 18:12,0.00,1313,0.00


In [35]:
#Used LabelEncoder to get numeric values
le = LabelEncoder()
X['VEHICLETYPE'] = le.fit_transform(X['VEHICLETYPE'])

X['SESSIONSTART'] = pd.to_datetime(X['SESSIONSTART'], format='%d/%m/%Y %H:%M', infer_datetime_format=True)
X['SESSIONEND'] = pd.to_datetime(X['SESSIONEND'], format='%d/%m/%Y %H:%M', infer_datetime_format=True)

X['SESSIONSTART'] = pd.to_datetime(X['SESSIONSTART']).map(pd.Timestamp.timestamp)
X['SESSIONEND'] = pd.to_datetime(X['SESSIONEND']).map(pd.Timestamp.timestamp)

X['SESSIONSTART'] = X['SESSIONSTART'] / 60
X['SESSIONEND'] = X['SESSIONEND'] / 60

print(X.head(10))
print(y.head(10))

#Standardized
sc = StandardScaler()
X = sc.fit_transform(X)

#Return back to dataFrame
X = pd.DataFrame(X)

       VEHICLETYPE  SESSIONSTART  SESSIONEND  TOTAL_CHARGE  DURATION  \
6771             0    27155360.0  27155421.0          1.22        61   
10923            0    27253188.0  27253208.0          0.00        20   
5242             0    27258936.0  27258937.0          0.00         1   
1336             0    27231781.0  27232407.0          0.00       626   
3978             0    27250339.0  27251652.0          0.00      1313   
7655             0    27179856.0  27180038.0          3.64       182   
9124             0    27221463.0  27221623.0          3.20       160   
414              0    27225804.0  27226627.0          0.00       823   
5046             0    27257482.0  27258232.0          0.00       750   
7283             0    27169851.0  27170009.0          3.16       158   

       EFFECTIVE_CHARGE  
6771               1.22  
10923              0.00  
5242               0.00  
1336               0.00  
3978               0.00  
7655               3.64  
9124               3.20  

In [36]:
#Tuning with Hyper Parameters

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

xgb_classifier = XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1,
                               colsample_bynode=1,
                               colsample_bytree=0.5,
                               enable_categorical=False,
                               eval_metric='mlogloss',
                               gamma=0.1,  
                               gpu_id=-1,
                               importance_type=None,
                               interaction_constraints='',
                               learning_rate=0.2,
                               max_delta_step=0,
                               max_depth=15,
                               min_child_weight=1,
                               monotone_constraints='()',
                               n_estimators=100,
                               n_jobs=12, num_parallel_tree=1,
                               predictor='auto',
                               random_state=0,
                               reg_alpha=0,
                               reg_lambda=1,
                               scale_pos_weight=1,
                               subsample=1,
                               tree_method='exact',
                               validate_parameters=1,
                               verbosity=None)

xgb_classifier.fit(X_train, y_train)
y_pred_test_tuned = xgb_classifier.predict(X_test)   
accuracy_score(y_test, y_pred_test_tuned)

0.9424242424242424

In [37]:
import pickle
# create an iterator object with write permission - model.pkl
with open('XGBoost_pkl_Latest2.pkl', 'wb') as files:
    pickle.dump(xgb_classifier, files)
    
# model = pickle.load(open('XGBoost_pkl_Latest2.pkl', 'rb'))
# model.score(X_test, y_test)


xgb_classifier.save_model('model.txt')

def modelselect():
    model = xgboost.XGBClassifier()
    model.load_model('model.txt')
    return model


featureList = [0, 27249480.0, 27249481.0,  0.00, 1, 0]

toNParray = np.asarray(featureList)
ReshapedArray = toNParray.reshape(1,-1)

modelselect().predict(ReshapedArray)


AttributeError: 'list' object has no attribute 'reshape'